In [1]:
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


In [25]:
def angle_cos(p0, p1, p2):

    d1, d2 = (p0-p1).astype('float'), (p2-p1).astype('float')

    return abs( np.dot(d1, d2) / np.sqrt( np.dot(d1, d1)*np.dot(d2, d2) ) )

def longueur_square (points):
    tab_long = []
    res = True
    for i in range(len(points)):
        if i <3:
            dist = np.sqrt((points[i+1][0]-points[i][0])**2 + (points[i+1][1]-points[i][1])**2)
        else:
            dist = np.sqrt((points[-1][0]-points[0][0])**2 + (points[-1][1]-points[0][1])**2)
        if dist < 70 or dist > 100:
            res = False

    return res

def find_squares(img):
    img = cv2.GaussianBlur(img, (5, 5), 0)
    squares = []
    for gray in cv2.split(img):
        for thrs in range(0, 255, 26):
            if thrs == 0:
                bin = cv2.Canny(gray, 0, 50, apertureSize=5)
                bin = cv2.dilate(bin, None)
            else:
                _retval, bin = cv2.threshold(gray, thrs, 255, cv2.THRESH_BINARY)
            bin, contours, _hierarchy = cv2.findContours(bin, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
            for cnt in contours:
                cnt_len = cv2.arcLength(cnt, True)
                cnt = cv2.approxPolyDP(cnt, 0.02*cnt_len, True)
                if len(cnt) == 4 and cv2.contourArea(cnt) > 1000 and cv2.isContourConvex(cnt):
                    cnt = cnt.reshape(-1, 2)
                    max_cos = np.max([angle_cos( cnt[i], cnt[(i+1) % 4], cnt[(i+2) % 4] ) for i in range(4)])
                    #print(cnt)
                    a = (cnt[1][1] - cnt[0][1])

                    if max_cos < 0.1 and a < img.shape[0]*0.8:

                        squares.append(cnt)
    return squares


In [2]:
imgobj = cv2.imread('img/green_face.jpg',0)
cv2.imshow('',imgobj)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [33]:
# import the opencv library
import cv2



vid = cv2.VideoCapture(0)

while(True):

    # Capture the video frame
    # by frame
    ret, frame = vid.read()


    grayFrame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(grayFrame, (3, 3), 0)
    canny = cv2.Canny(blurred, 20, 40)

    kernel = np.ones((3,3), np.uint8)
    dilated = cv2.dilate(canny, kernel, iterations=2)

    (contours, hierarchy) = cv2.findContours(dilated.copy(),
                                         cv2.RETR_TREE,
                                         cv2.CHAIN_APPROX_SIMPLE)
    squares = []
    #for each of the contours detected, the shape of the contours is approximated using approxPolyDP() function and the contours are drawn in the image using drawContours() function
    for count in contours:
        epsilon = 0.04 * cv2.arcLength(count, True)
        approximations = cv2.approxPolyDP(count, epsilon, True)
        if len(approximations) == 4 and cv2.isContourConvex(approximations):
            approximations = approximations.reshape(-1, 2)
            max_cos = np.max([angle_cos( approximations[i], approximations[(i+1) % 4], approximations[(i+2) % 4] ) for i in range(4)])
            if max_cos <0.2 and cv2.contourArea(approximations)>5000 and cv2.contourArea(approximations)<10000 and longueur_square(approximations):
                squares.append(approximations)

    cv2.drawContours(frame, squares, -1, (0,255,0), 3)

    #cv2.drawContours(frame,contours, -1, (0,255,0), 3)


    # Display the resulting frame
    cv2.imshow('frame', frame)


    # the 'q' button is set as the
    # quitting button you may use any
    # desired button of your choice
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# After the loop release the cap object
vid.release()

cv2.destroyAllWindows()

In [23]:
print(len(squares))
#print(squares)
for i in range (len(squares)):
    print(longueur_square(squares[i]))

9
False
True
False
False
True
True
False
False
True


In [8]:
imgobj = cv2.imread('img/mixed_face.jpg')
print(imgobj.shape)
grayFrame = cv2.cvtColor(imgobj, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(grayFrame, (3, 3), 0)
canny = cv2.Canny(blurred, 20, 40)

kernel = np.ones((3,3), np.uint8)
dilated = cv2.dilate(canny, kernel, iterations=2)
(contours, hierarchy) = cv2.findContours(dilated.copy(),
                                         cv2.RETR_TREE,
                                         cv2.CHAIN_APPROX_SIMPLE)

squares = []
#for each of the contours detected, the shape of the contours is approximated using approxPolyDP() function and the contours are drawn in the image using drawContours() function
for count in contours:
    epsilon = 0.04 * cv2.arcLength(count, True)
    approximations = cv2.approxPolyDP(count, epsilon, True)
    if len(approximations) == 4 and cv2.isContourConvex(approximations):
        approximations = approximations.reshape(-1, 2)
        max_cos = np.max([angle_cos( approximations[i], approximations[(i+1) % 4], approximations[(i+2) % 4] ) for i in range(4)])
        if max_cos <0.2 and cv2.contourArea(approximations)>5000 and cv2.contourArea(approximations)<10000:
            squares.append(approximations)

cv2.drawContours(imgobj, squares, -1, (0,255,0), 3)
cv2.imshow('',imgobj)

cv2.waitKey(0)
cv2.destroyAllWindows()
#print(len(squares))

(4032, 3024, 3)


In [112]:
def position(contours):
    pt_centre = []
    for i in range(len(contours)):
        pt_centre.append(np.mean(contours[i], axis = 0))

    pt_centre = np.array(pt_centre)
    ind_y = pt_centre[:,1].argsort()

    #Peut etre le mettre avant la fonction, jsp
    contours = np.array(contours)
    contours_sorted_y = contours[ind_y]
    centre_sorted_y = pt_centre[ind_y]

    pt_centre1 = centre_sorted_y[0:3]
    pt_centre2 = centre_sorted_y[3:6]
    pt_centre3 = centre_sorted_y[6:9]

    ligne1 = contours_sorted_y[0:3]
    ligne2 = contours_sorted_y[3:6]
    ligne3 = contours_sorted_y[6:9]

    ind_x1 = pt_centre1[:,0].argsort()
    ind_x2 = pt_centre2[:,0].argsort()
    ind_x3 = pt_centre3[:,0].argsort()

    ligne1 = ligne1[ind_x1]
    ligne2 = ligne2[ind_x2]
    ligne3 = ligne3[ind_x3]

    res = []
    res.append(ligne1)
    res.append(ligne2)
    res.append(ligne3)

    return res

In [115]:
print(squares)
print(position(squares))
"""pt_centre = []
for i in range(len(squares)):
        pt_centre.append(np.mean(squares[i], axis = 0))
pt_centre = np.array(pt_centre)

ind = pt_centre[pt_centre[:,1].argsort()]
print(ind[0:2])"""




[[[ 877  466]
  [ 888  549]
  [ 800  557]
  [ 793  471]]

 [[1007  462]
  [1017  550]
  [ 928  557]
  [ 918  468]]

 [[1136  459]
  [1148  549]
  [1059  556]
  [1048  467]]

 [[ 863  342]
  [ 873  425]
  [ 785  432]
  [ 778  350]]

 [[ 991  342]
  [ 999  427]
  [ 913  432]
  [ 904  349]]

 [[1122  337]
  [1131  420]
  [1042  428]
  [1032  344]]

 [[ 855  222]
  [ 864  300]
  [ 781  309]
  [ 773  231]]

 [[ 982  222]
  [ 990  299]
  [ 904  307]
  [ 899  226]]

 [[1110  213]
  [1120  295]
  [1034  304]
  [1022  223]]]
[array([[[ 855,  222],
        [ 864,  300],
        [ 781,  309],
        [ 773,  231]],

       [[ 982,  222],
        [ 990,  299],
        [ 904,  307],
        [ 899,  226]],

       [[1110,  213],
        [1120,  295],
        [1034,  304],
        [1022,  223]]], dtype=int32), array([[[ 863,  342],
        [ 873,  425],
        [ 785,  432],
        [ 778,  350]],

       [[ 991,  342],
        [ 999,  427],
        [ 913,  432],
        [ 904,  349]],

       [[1122

'pt_centre = []\nfor i in range(len(squares)):\n        pt_centre.append(np.mean(squares[i], axis = 0))\npt_centre = np.array(pt_centre)\n\nind = pt_centre[pt_centre[:,1].argsort()]\nprint(ind[0:2])'